## PDX - Phuture Decentralised Exchange Index - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  0.251e8
min_weight = 0.005
max_weight = 0.2
circ_supply_threshold = 0.3
liveness_threshold = 365 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-exchange"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')
liquid_staking_tokens = pd.DataFrame(cg.get_coins_markets('usd',category='liquid-staking-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens,liquid_staking_tokens]
    )
liquidity

Note: chex-token, marketcap data available only for 57 < 365 days
Note: quickswap, marketcap data available only for 328 < 365 days
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x081f67afa0ccf8c7b17540767bbe95df2ba8d97f ethereum
0x753d224bcf9aafacd81558c32341416df61d3dac arbitrum-one
0xd3594e879b358f430e20f82bea61e83562d49d48 optimistic-ethereum
Dropping dydx because a price feed is unavailable
Dropping paraswap because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
uniswap,0.165591,0.165591,5.589810e-08,arbitrum-one
pancakeswap-token,0.398423,0.397897,-1.319208e-03,binance-smart-chain
havven,0.322853,0.320312,-7.872240e-03,optimistic-ethereum
1inch,2.493282,2.466102,-1.090120e-02,ethereum
paraswap,26.155580,25.778364,-1.442200e-02,ethereum
quickswap,20.942418,20.637529,-1.455846e-02,polygon-pos
joe,2.141757,2.104571,-1.736193e-02,arbitrum-one
bancor,1.449001,1.416147,-2.267318e-02,ethereum
balancer,0.279918,0.271551,-2.989004e-02,ethereum


## Show Results

In [8]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
uniswap,UNI,4.535031e+09,0.200000,13108,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
havven,SNX,1.012430e+09,0.200000,13108,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,optimistic-ethereum
pancakeswap-token,CAKE,6.191912e+08,0.129210,8468,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
gnosis,GNO,5.581099e+08,0.116464,7633,0x6810e776880c02933d47db1b9fc05908e5386b96,ethereum
curve-dao-token,CRV,4.961610e+08,0.103536,6785,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
1inch,1INCH,4.499500e+08,0.093893,6153,0x111111111117dc0aa78b770fa6a738034120c302,ethereum
sushi,SUSHI,2.054255e+08,0.042867,2809,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,ethereum
balancer,BAL,1.982537e+08,0.041371,2711,0xba100000625a3754423978a60c9317c58a424e3d,ethereum
joe,JOE,1.584785e+08,0.033070,2167,0x371c7ec6d8039ff7933a2aa28eb827ffe1f52f07,arbitrum-one
